###### Load used modules

In [1]:
using BenchmarkTools
using MAT
using DataFrames
using SymPy
using Plots;
using DifferentialEquations
using DiffEqOperators
using DiffEqParamEstim
using DiffEqDevTools
using DiffEqSensitivity
using StaticArrays
using OrdinaryDiffEq
using LinearAlgebra
using ODE
using ODEInterface
using ODEInterfaceDiffEq
using LSODA
using ModelingToolkit
using SparsityDetection
using SparseArrays
using Sundials
using ParameterizedFunctions
using PolynomialRoots
using Polynomials
using Roots
LinearAlgebra.BLAS.set_num_threads(1)

###### Set input parameters, models and ICs

In [2]:
# Switch for model oscillator: 1 - anharmonic; 2 - harmonic
const sw_o   = 1;
# const sw_o = 2;

const sw_arr   = "Park";
# const sw_arr = "Scanlon";

const sw_u   = "D/6k";
# const sw_u = "3T";
# const sw_u = "Inf";

const sw_z   = "Savelev";
# const sw_z = "Starik";
# const sw_z = "Stellar";

const c     = 2.99e8;
const h     = 6.6261e-34;
const k     = 1.3807e-23;
const N_a   = 6.0221e23;
const h_bar = h/(2*pi);
const R     = 8.3145;

# N2-O2-NO, i^-1
const om_e   = [235857 158019 190420];
const om_x_e = [1432 1198 1407.5];
const Be     = [1.998 1.4377 1.6720]*100;

# load("wurster_data_times.mat");
# data_40_60 = sortrows(data_40_60);
# data_22_77 = sortrows(data_22_77);
# data_5_95 = sortrows(data_5_95);
# data_exp = [data_5_95; data_22_77; data_40_60];

if (sw_arr == "Park")

  #const ARR_D_data = readtable("arr_d_park.dat");
  #const ARR_Z_data = readtable("arr_z_park.dat");

  #const D  = ARR_D_data([3,6,9],1)';
  #const D  = [113200  113200  113200  113200  113200;
  #            59500 59500 59500 59500 59500;
  #            75500 75500 75500 75500 75500];
  const D  = [113200; 59500; 75500];
  println("D = ", D, "\n", size(D), "\n");
  #const CA = ARR_D_data([1,4,7],:);
  const CA = [1.162385214460072e-08 1.162385214460072e-08 1.162385214460072e-08 4.981650919114595e-08 4.981650919114595e-08;
              3.321100612743063e-09 3.321100612743063e-09 3.321100612743063e-09 1.660550306371531e-08 1.660550306371531e-08;
              8.302751531857657e-15 8.302751531857657e-15 1.826605337008685e-13 1.826605337008685e-13 1.826605337008685e-13];
  println("CA = ", CA, "\n", size(CA), "\n");
  #const nA = ARR_D_data([2,5,8],:);
  const nA = [-1.6  -1.6  -1.6  -1.6  -1.6;
              -1.5  -1.5  -1.5  -1.5  -1.5;
              0 0 0 0 0];
  println("nA = ", nA, "\n", size(nA), "\n");

  #const CAZ = ARR_Z_data(:,1);
  #const nAZ = ARR_Z_data(:,2);
  #const EaZ = ARR_Z_data(:,3);
  const CAZ = [1.06274e-12 1.39485e-17]; # [m^3/sec]
  const nAZ = [-1. 0.];
  const EaZ = [5.29756e-19 2.68537e-19]; # [J]

elseif (sw_arr == "Scanlon")

  # NO DATA
  #const ARR_D_data = readtable("arr_d_scanlon.dat");
  #const ARR_Z_data = readtable("arr_z_scanlon.dat");

  #const D  = ARR_D_data([3,6,9],1)'/k; # J->K
  #const CA = ARR_D_data([1,4,7],:); # m^3/sec
  #const nA = ARR_D_data([2,5,8],:);

  #const CAZ = ARR_Z_data(:,1); # m^3/sec
  #const nAZ = ARR_Z_data(:,2);
  #const EaZ = ARR_Z_data(:,3);

else
  disp("Error! Check Arrhenius input.")
  return;
end

const QN = [47 36 39; 33 26 28];
const l = QN[sw_o,:];

D = [113200, 59500, 75500]
(3,)

CA = [1.162385214460072e-8 1.162385214460072e-8 1.162385214460072e-8 4.981650919114595e-8 4.981650919114595e-8; 3.321100612743063e-9 3.321100612743063e-9 3.321100612743063e-9 1.660550306371531e-8 1.660550306371531e-8; 8.302751531857657e-15 8.302751531857657e-15 1.826605337008685e-13 1.826605337008685e-13 1.826605337008685e-13]
(3, 5)

nA = [-1.6 -1.6 -1.6 -1.6 -1.6; -1.5 -1.5 -1.5 -1.5 -1.5; 0.0 0.0 0.0 0.0 0.0]
(3, 5)



###### Compute the vibrational energy for each molecular species

In [3]:
function en_vibr(sp)

if sw_o == 1
  om_0 = om_e[sp]-om_x_e[sp];
  om_x_e_0 = om_x_e[sp];
  e = h*c*(om_0*(0:l[sp]-1)-om_x_e_0*((0:l[sp]-1)).^2);
else
  e = h*c*om_e[sp]*(0:l[sp]-1);
end

end

en_vibr (generic function with 1 method)

###### Compute the ground vibrational energy level for each molecular species

In [4]:
function en_vibr_0(sp)

if sw_o == 1
  e = h*c*(0.5*om_e[sp]-0.25*om_x_e[sp]);
else
  e = h*c*0.5*om_e[sp];
end

end

en_vibr_0 (generic function with 1 method)

In [5]:
const en2_i = en_vibr(1);   println("en2_i = ", en2_i, "\n", size(en2_i), "\n")
const en2_0 = en_vibr_0(1); println("en2_0 = ", en2_0, "\n", size(en2_0), "\n")
const eo2_i = en_vibr(2);   println("eo2_i = ", eo2_i, "\n", size(eo2_i), "\n")
const eo2_0 = en_vibr_0(2); println("eo2_0 = ", eo2_0, "\n", size(eo2_0), "\n")
const eno_i = en_vibr(3);   println("eno_i = ", eno_i, "\n", size(eno_i), "\n")
const eno_0 = en_vibr_0(3); println("eno_0 = ", eno_0, "\n", size(eno_0), "\n")

const ef = [0., 0., (0.5*(D[1]+D[2])-D[3]), 0.5*D[1], 0.5*D[2]]*k;

const mu = [28 32 30 14 16]*1e-3; # kg/mol
const m = mu./N_a; # N2-O2-NO-N-O

# N2-O2-NO
const osc_mass = [0.5*m[4], 0.5*m[5], m[4]*m[5]/(m[4]+m[5])];

# mass(atom1)/mass(molecule), mass(atom2)/mass(molecule)
const ram_masses = [0.5 0.5;        # 'N2'
                    0.5 0.5;        # 'O2'
                    0.4668 0.5332]; # 'NO'
println("ram_masses = ", ram_masses, "\n", size(ram_masses), "\n");

const fho_data_n2 = [3.9e10  1   * k  0.9;    # 'N2+N2'
                     3.9e10  6   * k  0.95;   # 'N2+O2'
                     4e10    2   * k  0.75;   # 'N2+NO'
                     4.6e10  1   * k  0.99;   # 'N2+N'
                     7.3e10  500 * k  0.175]; # 'N2+O'
println("fho_data_n2 = ", fho_data_n2, "\n", size(fho_data_n2), "\n");

const fho_data_o2 = [4.1e10  150   * k  0.333333; # 'O2+N2'
                     4.3e10  40    * k  0.99;     # 'O2+O2'
                     4.1e10  150   * k  0.333333; # 'O2+NO'
                     7.3e10  10    * k  0.25;     # 'O2+N'
                     2.6e10  17000 * k  0.2];     # 'O2+O'
println("fho_data_o2 = ", fho_data_o2, "\n", size(fho_data_o2), "\n");

const fho_data_no = [4.4e10  20 * k  0.9;      # 'NO+N2'
                     6.75e10  1500 * k  0.2;   # 'NO+O2'
                     6.25e10  4500 * k  0.03;  # 'NO+NO'
                     5e10  200 * k  0.3183;    # 'NO+N'
                     7.9e10  16000 * k  0.06]; # 'NO+O'
println("fho_data_no = ", fho_data_no, "\n", size(fho_data_no), "\n");

const FHO_data = cat(fho_data_n2,fho_data_o2,fho_data_no, dims=3);
println("FHO_data = ", FHO_data, "\n", size(FHO_data), "\n");

# VSS: dref [i], omega
const vss_data_n2 = [4.04e-10 0.686;                # 'N2+N2'
                     3.604e-10 0.703;               # 'N2+O2'
                     4.391e-10 0.756;               # 'N2+NO'
                     4.088e-10 0.762;               # 'N2+N'
                     3.2220000000000004e-10 0.702]; # 'N2+O'
println("vss_data_n2 = ", vss_data_n2, "\n", size(vss_data_n2), "\n");

const vss_data_o2 = [3.604e-10 0.703;              # 'O2+N2'
                     3.8960000000000003e-10 0.7;   # 'O2+O2'
                     4.054e-10 0.718;              # 'O2+NO'
                     3.7210000000000004e-10 0.757; # 'O2+N'
                     3.734e-10 0.76];              # 'O2+O'
println("vss_data_o2 = ", vss_data_o2, "\n", size(vss_data_o2), "\n");

const vss_data_no = [4.391e-10 0.756;              # 'NO+N2'
                     4.054e-10 0.718;              # 'NO+O2'
                     4.2180000000000003e-10 0.737; # 'NO+NO'
                     4.028e-10 0.788;              # 'NO+N'
                     3.693e-10 0.752];             # 'NO+O'
println("vss_data_no = ", vss_data_no, "\n", size(vss_data_no), "\n");

const vss_data = cat(vss_data_n2,vss_data_o2,vss_data_no, dims=3);
println("vss_data = ", vss_data, "\n", size(vss_data), "\n");

# N2-N2, O2-O2, NO-NO, N-N, O-O
const R0 = [3.621e-10 3.458e-10 3.47e-10 3.298e-10 2.75e-10]; # m

# N2-N2, O2-O2, NO-NO, N-N, O-O
const EM = [97.5; 107.4; 119; 71.4; 80]; # K
println("EM = ", EM, "\n", size(EM), "\n");

const r0 = 0.5 .* [R0[1].+R0; R0[2].+R0; R0[3].+R0]; println("r0 = ", r0, "\n", size(r0), "\n")

# unused and wrong dimensions
#const em = [sqrt.(EM[1]*R0[1]^6 * EM.*R0.^6)./r0[1,:].^6;
#            sqrt.(EM[2]*R0[2]^6 * EM.*R0.^6)./r0[2,:].^6;
#            sqrt.(EM[3]*R0[3]^6 * EM.*R0.^6)./r0[3,:].^6];
#println("em = ", em, "\n", size(em), "\n");

const re = [1.097; 1.207; 1.151] .* 1e-10; # N2-O2-NO

# Wurster, 1991
const us_95_5_exp  = [3.87; 3.49; 3.15; 2.97];
const us_78_22_exp = [3.85; 3.52; 3.26; 2.99];
const us_60_40_exp = [3.85; 3.47; 3.24; 3.06];

# xN2-xO2-us
const incon = [0.95; 0.05; us_95_5_exp[1]];
#const incon = [0.777, 0.223, us_78_22_exp[1]];
#const incon = [0.6, 0.4, us_60_40_exp[1]];

const Torr = 133.322;
const p0 = 2.25*Torr; # Pa
const T0 = 300.;      # K
const Tv0n2 = T0;     # K
const Tv0o2 = T0;     # K
const Tv0no = T0;     # K

const Zv0_n2 = sum(exp.(-en2_i./Tv0n2/k));
const Zv0_o2 = sum(exp.(-eo2_i./Tv0o2/k));
const Zv0_no = sum(exp.(-eno_i./Tv0no/k));
println("Zv0_n2 = ", Zv0_n2, "\n");
println("Zv0_o2 = ", Zv0_o2, "\n");
println("Zv0_no = ", Zv0_no, "\n");

const n0 = p0/(k*T0);
println("n0 = ", n0, "\n")

# N2-N2
const sigma0 = pi*R0[1]^2;
const lambda0 = 1/(sqrt(2)*n0*sigma0);
const Delta = lambda0;
println("Delta = ", Delta, "\n", size(Delta), "\n");

# xc = nc/n
const xc = zeros(5);
const xc[1] = incon[1];
const xc[2] = incon[2];
const v0    = incon[3]*1e3;

const gamma0 = 1.4;
const rho0_c = m.*xc*n0;
const rho0 = sum(rho0_c);
const mu0_mix = sum(rho0_c./mu)/rho0;
const R_bar = R*mu0_mix;
const a0 = sqrt(gamma0*R_bar*T0);
const M0 = v0/a0;

#NN = in_con(); FIXME
NN = [5.761793697284453e+00 1.735570644383367e-01 2.445965060176968e+01]
n1 = NN[1]; println("n1 = ", n1, "\n")
v1 = NN[2]; println("v1 = ", v1, "\n")
T1 = NN[3]; println("T1 = ", T1, "\n")

Y0_bar = zeros(sum(l)+4);

Y0_bar[1:l[1]] = xc[1]*n1/Zv0_n2*exp.(-en2_i./Tv0n2/k);             # N2
Y0_bar[l[1]+1:l[1]+l[2]] = xc[2]*n1/Zv0_o2*exp.(-eo2_i./Tv0o2/k);   # O2
Y0_bar[l[1]+l[2]+1:sum(l)] = xc[3]*n1/Zv0_no*exp.(-eno_i./Tv0no/k); # NO
Y0_bar[sum(l)+1] = xc[4];                                           # N
Y0_bar[sum(l)+2] = xc[5];                                           # O
Y0_bar[sum(l)+3] = v1;
Y0_bar[sum(l)+4] = T1;
println("Y0_bar = ", Y0_bar, "\n", size(Y0_bar), "\n");

const x_w = 2.; # m
const xspan = [0., x_w]./Delta;

en2_i = [0.0, 4.616066402727e-20, 9.175391125758e-20, 1.3677974169093e-19, 1.8123815532732e-19, 2.2512915216675e-19, 2.6845273220922003e-19, 3.1120889545473e-19, 3.5339764190328e-19, 3.9501897155487e-19, 4.360728844095e-19, 4.7655938046717e-19, 5.1647845972788e-19, 5.5583012219163e-19, 5.9461436785842e-19, 6.3283119672825e-19, 6.7048060880112e-19, 7.0756260407703e-19, 7.4407718255598e-19, 7.8002434423797e-19, 8.15404089123e-19, 8.502164172110701e-19, 8.844613285021801e-19, 9.181388229963301e-19, 9.5124890069352e-19, 9.8379156159375e-19, 1.01576680569702e-18, 1.0471746330033301e-18, 1.07801504351268e-18, 1.10828803722507e-18, 1.1379936141405e-18, 1.16713177425897e-18, 1.19570251758048e-18, 1.22370584410503e-18, 1.2511417538326201e-18, 1.27801024676325e-18, 1.30431132289692e-18, 1.33004498223363e-18, 1.35521122477338e-18, 1.3798100505161701e-18, 1.403841459462e-18, 1.42730545161087e-18, 1.45020202696278e-18, 1.47253118551773e-18, 1.49429292727572e-18, 1.51548725223675e-18, 1.536114160400

###### Define the r.h.s source term function

In [6]:
function rpart_fho!(dy, y, t, p)

nn2i_b = zeros(l[1])
no2i_b = zeros(l[2])
nnoi_b = zeros(l[3])

l1    = l[1];
l2    = l[2];
l3    = l[3];
lall  = l1+l2+l3;
Lmax1 = l1-1;
Lmax2 = l2-1;
Lmax3 = l3-1;

nn2i_b = y[1:l1];
no2i_b = y[l1+1:l1+l2];
nnoi_b = y[l1+l2+1:lall];
nn_b   = y[lall+1];
no_b   = y[lall+2];
v_b    = y[lall+3];
T_b    = y[lall+4];

nn2_b = sum(nn2i_b);
no2_b = sum(no2i_b);
nno_b = sum(nnoi_b);

na_b = nn_b+no_b;
nm_b = sum(y[1:lall]);

temp = T_b*T0;

efn_b = 0.5*D[1]/T0;
efo_b = 0.5*D[2]/T0;
efno_b = efn_b+efo_b-D[3]/T0;

en2i_b = en2_i/(k*T0);
en20_b = en2_0/(k*T0);
eo2i_b = eo2_i/(k*T0);
eo20_b = eo2_0/(k*T0);
enoi_b = eno_i/(k*T0);
eno0_b = eno_0/(k*T0);

sigma = [2 2 1];
Theta_r = Be*h*c/k;
Z_rot = temp./(sigma.*Theta_r);
println("Z_rot = ", Z_rot, "\n")

M = sum(m);
mb = m/M;

# A*X=B
A = zeros(sum(l)+4,sum(l)+4);

for i=1:l1 # N2
  A[i,i] = v_b;
  A[i,lall + 3] = nn2i_b[i];
end
for i=1:l2 # O2
  A[l1+i,l1+i] = v_b;
  A[l1+i,lall + 3] = no2i_b[i];
end
for i=1:l3 # NO
  A[l1+l2+i,l1+l2+i] = v_b;
  A[l1+l2+i,lall + 3] = nnoi_b[i];
end

# N
A[lall+1,lall+1] = v_b;
A[lall+1,lall + 3] = nn_b;
# O
A[lall+2,lall+2] = v_b;
A[lall+2,lall + 3] = no_b;

for i=1:lall+2
  A[lall+3,i] = T_b;
end
A[lall+3,lall+3] = M*v0^2/k/T0*(mb[1]*nn2_b+mb[2]*no2_b+mb[3]*nno_b+mb[4]*nn_b+mb[5]*no_b)*v_b;
A[lall+3,lall+4] = nm_b+na_b;

for i=1:l1 # N2
  A[lall+4,i] = 2.5*T_b+en2i_b[i]+en20_b;
end
for i=1:l2 # O2
  A[lall+4,l1+i] = 2.5*T_b+eo2i_b[i]+eo20_b;
end
for i=1:l3 # NO
  A[lall+4,l1+l2+i] = 2.5*T_b+enoi_b[i]+eno0_b+efno_b;
end
A[lall+4,lall+1] = 1.5*T_b+efn_b; # N
A[lall+4,lall+2] = 1.5*T_b+efo_b; # O
A[lall+4,lall+3] = 1/v_b*(3.5*nm_b*T_b+2.5*na_b*T_b + sum((en2i_b.+en20_b).*nn2i_b)
                                                    + sum((eo2i_b.+eo20_b).*no2i_b)
                                                    + sum((enoi_b.+eno0_b).*nnoi_b)
                                                    + efno_b*nno_b
                                                    + efn_b *nn_b
                                                    + efo_b *no_b);
A[lall+4,lall+4] = 2.5*nm_b+1.5*na_b;
AA = inv(A);
#println("AA = ", AA, "\n", size(AA), "\n")
spy(AA, markersize=1, colorbar=false, color=:deep)

# Dissociation/Recombination processes
# k_rec / k_di
Kdr_n2 = (m[1]*h^2/(m[4]*m[4]*2*pi*k*temp))^(3/2)*Z_rot[1]* exp.(-en2_i'/(k*temp))*exp(D[1]/temp);
Kdr_o2 = (m[2]*h^2/(m[5]*m[5]*2*pi*k*temp))^(3/2)*Z_rot[2]* exp.(-eo2_i'/(k*temp))*exp(D[2]/temp);
Kdr_no = (m[3]*h^2/(m[4]*m[5]*2*pi*k*temp))^(3/2)*Z_rot[3]* exp.(-eno_i'/(k*temp))*exp(D[3]/temp);

println("Kdr_n2 = ", Kdr_n2, "\n", size(Kdr_n2), "\n")
println("Kdr_o2 = ", Kdr_o2, "\n", size(Kdr_o2), "\n")
println("Kdr_no = ", Kdr_no, "\n", size(Kdr_no), "\n")

kd_n2 = zeros(5,l1);
kd_o2 = zeros(5,l2);
kd_no = zeros(5,l3);
kr_n2 = zeros(5,l1);
kr_o2 = zeros(5,l2);
kr_no = zeros(5,l3);

function kdis(t,sp)

# equil. coef-s
kd_eq = CA[sp,:].*t.^nA[sp,:]*exp(-D[sp]/t); # m^3/sec

# parameter of TM model
if sw_u == "D/6k"
  U = D[sp]/6;
elseif sw_u == "3T"
  U = 3*t;
elseif sw_u == "Inf"
  U = Inf;
else
  disp("Error! Check switch on parameter U.");
  return;
end

# U = Inf;

#equil. vibr. partition function
if sp == 1

  ZvT = sum(exp.(-en2_i./(t*k)));
  ZvU = sum(exp.(en2_i./(U*k)));

  # non-equilibrium factor
  Z = ZvT / ZvU * exp.(en2_i./k*(1/t + 1/U));

elseif sp == 2

    ZvT = sum(exp.(-eo2_i./(t*k)));
    ZvU = sum(exp.(eo2_i./(U*k)));

    #non-equilibrium factor
    Z = ZvT / ZvU * exp.(eo2_i./k*(1/t + 1/U));

elseif sp == 3

    ZvT = sum(exp.(-eno_i./(t*k)));
    ZvU = sum(exp.(eno_i./(U*k)));

    # non-equilibrium factor
    Z = ZvT / ZvU * exp.(eno_i./k*(1/t + 1/U));

end

# dis. rates
kd = kd_eq .* Z'; # m^3/sec
println("kd = ", kd, "\n", size(kd), "\n", typeof(kd), "\n")
return kd
end

kd_n2 = kdis(temp,1) * Delta*n0/v0;
kd_o2 = kdis(temp,2) * Delta*n0/v0;
kd_no = kdis(temp,3) * Delta*n0/v0;

for iM = 1:5
  kr_n2[iM,:] = kd_n2[iM,:] .* Kdr_n2 * n0;
  kr_o2[iM,:] = kd_o2[iM,:] .* Kdr_o2 * n0;
  kr_no[iM,:] = kd_no[iM,:] .* Kdr_no * n0;
end

# Exchange processes
kf_n2 = zeros(l1,l3)
kb_n2 = zeros(l1,l3)
kf_o2 = zeros(l2,l3)
kb_o2 = zeros(l2,l3)

# kb_exchange / kf_exchange
# https://discourse.julialang.org/t/julia-1-0-2-reapeat-function-cost-many-times-than-matlab-s-repmat/17708/3
# https://stackoverflow.com/questions/24846899/tiling-or-repeating-n-dimensional-arrays-in-julia
#Kz_n2 = (m[1]*m[5]/(m[3]*m[4]))^1.5*Z_rot[1]/Z_rot[3]* exp.((repeat(eno_i',l[1],1)-repeat(en2_i,1,l[3]))/(k*temp))*exp((D[1]-D[3])/temp);
#Kz_o2 = (m[2]*m[4]/(m[3]*m[5]))^1.5*Z_rot[2]/Z_rot[3]* exp.((repeat(eno_i',l[2],1)-repeat(eo2_i,1,l[3]))/(k*temp))*exp((D[2]-D[3])/temp);
#println("Kz_n2 = ", Kz_n2, "\n", size(Kz_n2), "\n")
#println("Kz_o2 = ", Kz_o2, "\n", size(Kz_o2), "\n")
# repeat((@view a1[i,:])',outer = (clen-i+1,1));

# kb_VT(i-1->i) / kf_VT(i->i-1)
#Kvt_n2 = exp.((en2_i[1:end-1]-en2_i[2:end])/(k*temp));
#Kvt_o2 = exp.((eo2_i[1:end-1]-eo2_i[2:end])/(k*temp));
#Kvt_no = exp.((eno_i[1:end-1]-eno_i[2:end])/(k*temp));
#println("Kvt_n2 = ", Kvt_n2, "\n", size(Kvt_n2), "\n")
#println("Kvt_o2 = ", Kvt_o2, "\n", size(Kvt_o2), "\n")
#println("Kvt_no = ", Kvt_no, "\n", size(Kvt_no), "\n")

#if sw_z == "Savelev"
#
#  # STELLAR database
#  kf_n2, kf_o2 = k_ex_savelev_st(temp);
#  kf_n2 = kf_n2 * Delta*n0/v0;
#  kf_o2 = kf_o2 * Delta*n0/v0;
#  kb_n2 = kf_n2 .* Kz_n2;
#  kb_o2 = kf_o2 .* Kz_o2;
#
#elseif sw_z == "Starik"
#
#  # NO DATA
#  kf_n2, kb_o2 = k_ex_starik(temp);
#  kf_n2 = kf_n2 * Delta*n0/v0;
#  kb_o2 = kb_o2 * Delta*n0/v0;
#  kb_n2 = kf_n2 .* Kz_n2;
#  kf_o2 = kb_o2 ./ Kz_o2;
#
#elseif sw_z == "Stellar"
#
#    # STELLAR database -- NO DATA
#    kf_n2, kf_o2 = k_ex_st_interp(temp);
#    kf_n2 = kf_n2 * Delta*n0/v0;
#    kf_o2 = kf_o2 * Delta*n0/v0;
#    kb_n2 = kf_n2 .* Kz_n2;
#    kb_o2 = kf_o2 .* Kz_o2;
#
#else
#
#  println("Error! Check switch on Zeldovich reaction model.");
#  return;
#
#end

# VT: i+1 -> i
kvt_down_n2 = zeros(5,Lmax1);
kvt_down_o2 = zeros(5,Lmax2);
kvt_down_no = zeros(5,Lmax3);
#for iM = 1:5
#  for i1 = 1:Lmax1
#    kvt_down_n2[iM,i1] = kvt_fho(1, iM, temp, i1, i1-1) * Delta*n0/v0;
#  end
#  for i2 = 1:Lmax2
#    kvt_down_o2[iM,i2] = kvt_fho(2, iM, temp, i2, i2-1) * Delta*n0/v0;
#  end
#  for i3 = 1:Lmax3
#    kvt_down_no[iM,i3] = kvt_fho(3, iM, temp, i3, i3-1) * Delta*n0/v0;
#  end
#end
#
## VT: i -> i+1
kvt_up_n2 = zeros(5,Lmax1);
kvt_up_o2 = zeros(5,Lmax2);
kvt_up_no = zeros(5,Lmax3);
#for ip = 1:5
#  kvt_up_n2[ip,:] = kvt_down_n2[ip,:] .* Kvt_n2;
#  kvt_up_o2[ip,:] = kvt_down_o2[ip,:] .* Kvt_o2;
#  kvt_up_no[ip,:] = kvt_down_no[ip,:] .* Kvt_no;
#end

# VV: (i,j)->(i-1,j+1)
kvv_down_n2 = zeros(Lmax1,Lmax1);
kvv_down_o2 = zeros(Lmax2,Lmax2);
kvv_down_no = zeros(Lmax3,Lmax3);
#for i = 1:Lmax1
#  for j = 1:Lmax1
#    kvv_down_n2[i,j] = kvv_fho(1, 1, temp, i, j-1, i-1, j) * Delta*n0/v0;
#  end
#end
#for i = 1:Lmax2
#  for j = 1:Lmax2
#    kvv_down_o2[i,j] = kvv_fho(2, 2, temp, i, j-1, i-1, j) * Delta*n0/v0;
#  end
#end
#for i = 1:Lmax3
#  for j = 1:Lmax3
#    kvv_down_no[i,j] = kvv_fho(3, 3, temp, i, j-1, i-1, j) * Delta*n0/v0;
#  end
#end

# VV: (i-1,j+1)->(i,j)
kvv_up_n2 = zeros(Lmax1,Lmax1);
kvv_up_o2 = zeros(Lmax2,Lmax2);
kvv_up_no = zeros(Lmax3,Lmax3);
#deps_n2 = en2_i[1:end-1]-en2_i[2:end];
#deps_o2 = eo2_i[1:end-1]-eo2_i[2:end];
#deps_no = eno_i[1:end-1]-eno_i[2:end];
#for ip = 1:Lmax1
#  kvv_up_n2[ip,:] = kvv_down_n2[ip,:] .* exp.((deps_n2[ip].-deps_n2) / (k*temp));
#end
#for ip = 1:Lmax2
#  kvv_up_o2[ip,:] = kvv_down_o2[ip,:] .* exp.((deps_o2[ip].-deps_o2) / (k*temp));
#end
#for ip = 1:Lmax3
#  kvv_up_no[ip,:] = kvv_down_no[ip,:] .* exp.((deps_no[ip].-deps_no) / (k*temp));
#end

# VV': (i,j)->(i-1,j+1)
kvvs_d_n2_o2 = zeros(Lmax1,Lmax2);
kvvs_d_n2_no = zeros(Lmax1,Lmax3);
kvvs_d_o2_n2 = zeros(Lmax2,Lmax1);
kvvs_d_o2_no = zeros(Lmax2,Lmax3);
kvvs_d_no_n2 = zeros(Lmax3,Lmax1);
kvvs_d_no_o2 = zeros(Lmax3,Lmax2);
#for i = 1:Lmax1
#  for j = 1:Lmax2
#    kvvs_d_n2_o2[i,j] = kvv_fho(1, 2, temp, i, j-1, i-1, j) * Delta*n0/v0;
#  end
#  for j = 1:Lmax3
#    kvvs_d_n2_no[i,j] = kvv_fho(1, 3, temp, i, j-1, i-1, j) * Delta*n0/v0;
#  end
#end
#for i = 1:Lmax2
#  for j = 1:Lmax1
#    kvvs_d_o2_n2[i,j] = kvv_fho(2, 1, temp, i, j-1, i-1, j) * Delta*n0/v0;
#  end
#  for j = 1:Lmax3
#    kvvs_d_o2_no[i,j] = kvv_fho(2, 3, temp, i, j-1, i-1, j) * Delta*n0/v0;
#  end
#end
#for i = 1:Lmax3
#  for j = 1:Lmax1
#    kvvs_d_no_n2[i,j] = kvv_fho(3, 1, temp, i, j-1, i-1, j) * Delta*n0/v0;
#  end
#  for j = 1:Lmax2
#    kvvs_d_no_o2[i,j] = kvv_fho(3, 2, temp, i, j-1, i-1, j) * Delta*n0/v0;
#  end
#end

# VV': (i-1,j+1)->(i,j)
kvvs_u_n2_o2 = zeros(Lmax1,Lmax2);
kvvs_u_n2_no = zeros(Lmax1,Lmax3);
kvvs_u_o2_n2 = zeros(Lmax2,Lmax1);
kvvs_u_o2_no = zeros(Lmax2,Lmax3);
kvvs_u_no_n2 = zeros(Lmax3,Lmax1);
kvvs_u_no_o2 = zeros(Lmax3,Lmax2);
#for ip = 1:Lmax1
#  kvvs_u_n2_o2[ip,:] = kvvs_d_n2_o2[ip,:] .* exp.((deps_n2[ip].-deps_o2) / (k*temp));
#  kvvs_u_n2_no[ip,:] = kvvs_d_n2_no[ip,:] .* exp.((deps_n2[ip].-deps_no) / (k*temp));
#end
#for ip = 1:Lmax2
#  kvvs_u_o2_n2[ip,:] = kvvs_d_o2_n2[ip,:] .* exp.((deps_o2[ip].-deps_n2) / (k*temp));
#  kvvs_u_o2_no[ip,:] = kvvs_d_o2_no[ip,:] .* exp.((deps_o2[ip].-deps_no) / (k*temp));
#end
#for ip = 1:Lmax3
#  kvvs_u_no_n2[ip,:] = kvvs_d_no_n2[ip,:] .* exp.((deps_no[ip].-deps_n2) / (k*temp));
#  kvvs_u_no_o2[ip,:] = kvvs_d_no_o2[ip,:] .* exp.((deps_no[ip].-deps_o2) / (k*temp));
#end

RDn2   = zeros(l1); RDo2   = zeros(l2); RDno   = zeros(l3);
RZn2   = zeros(l1); RZo2   = zeros(l2); RZno   = zeros(l3);
RVTn2  = zeros(l1); RVTo2  = zeros(l2); RVTno  = zeros(l3);
RVVn2  = zeros(l1); RVVo2  = zeros(l2); RVVno  = zeros(l3);
RVVsn2 = zeros(l1); RVVso2 = zeros(l2); RVVsno = zeros(l3);

for i1 = 1:l1

  RDn2[i1] = nn2_b*(nn_b*nn_b*kr_n2[1,i1]-nn2i_b[i1]*kd_n2[1,i1]) +
             no2_b*(nn_b*nn_b*kr_n2[2,i1]-nn2i_b[i1]*kd_n2[2,i1]) +
             nno_b*(nn_b*nn_b*kr_n2[3,i1]-nn2i_b[i1]*kd_n2[3,i1]) +
             nn_b *(nn_b*nn_b*kr_n2[4,i1]-nn2i_b[i1]*kd_n2[4,i1]) +
             no_b *(nn_b*nn_b*kr_n2[5,i1]-nn2i_b[i1]*kd_n2[5,i1]);

  RZn2[i1] = sum(nnoi_b*nn_b.*kb_n2[i1,:] - nn2i_b[i1]*no_b*kf_n2[i1,:]);

  if i1 == 1 # 0<->1

    RVTn2[i1] = nn2_b*(nn2i_b[i1+1]*kvt_down_n2[1,i1] - nn2i_b[i1]*kvt_up_n2[1,i1])+
                no2_b*(nn2i_b[i1+1]*kvt_down_n2[2,i1] - nn2i_b[i1]*kvt_up_n2[2,i1])+
                nno_b*(nn2i_b[i1+1]*kvt_down_n2[3,i1] - nn2i_b[i1]*kvt_up_n2[3,i1])+
                nn_b *(nn2i_b[i1+1]*kvt_down_n2[4,i1] - nn2i_b[i1]*kvt_up_n2[4,i1])+
                no_b *(nn2i_b[i1+1]*kvt_down_n2[5,i1] - nn2i_b[i1]*kvt_up_n2[5,i1]);

    RVVn2[i1] = nn2i_b[i1+1]*sum(nn2i_b[1:end-1] .* kvv_down_n2[i1,:]) -
                nn2i_b[i1]  *sum(nn2i_b[2:end]   .* kvv_up_n2[i1,:]);

    RVVsn2[i1] = nn2i_b[i1+1]*(sum(no2i_b[1:end-1] .* kvvs_d_n2_o2[i1,:])) -
                 nn2i_b[i1]  *(sum(no2i_b[2:end]   .* kvvs_u_n2_o2[i1,:])) +
                 nn2i_b[i1+1]*(sum(nnoi_b[1:end-1] .* kvvs_d_n2_no[i1,:])) -
                 nn2i_b[i1]  *(sum(nnoi_b[2:end]   .* kvvs_u_n2_no[i1,:]));

  elseif i1 == l1 # Lmax <-> Lmax-1

    RVTn2[i1] = nn2_b*(nn2i_b[i1-1]*kvt_up_n2[1,i1-1] - nn2i_b[i1]*kvt_down_n2[1,i1-1])+
                no2_b*(nn2i_b[i1-1]*kvt_up_n2[2,i1-1] - nn2i_b[i1]*kvt_down_n2[2,i1-1])+
                nno_b*(nn2i_b[i1-1]*kvt_up_n2[3,i1-1] - nn2i_b[i1]*kvt_down_n2[3,i1-1])+
                nn_b *(nn2i_b[i1-1]*kvt_up_n2[4,i1-1] - nn2i_b[i1]*kvt_down_n2[4,i1-1])+
                no_b *(nn2i_b[i1-1]*kvt_up_n2[5,i1-1] - nn2i_b[i1]*kvt_down_n2[5,i1-1]);

    RVVn2[i1] = nn2i_b[i1-1]*sum(nn2i_b[2:end]   .* kvv_up_n2[i1-1,:]) -
                nn2i_b[i1]  *sum(nn2i_b[1:end-1] .* kvv_down_n2[i1-1,:]);

    RVVsn2[i1] = nn2i_b[i1-1]*(sum(no2i_b[2:end]   .* kvvs_u_n2_o2[i1-1,:])) -
                 nn2i_b[i1]  *(sum(no2i_b[1:end-1] .* kvvs_d_n2_o2[i1-1,:])) +
                 nn2i_b[i1-1]*(sum(nnoi_b[2:end]   .* kvvs_u_n2_no[i1-1,:])) -
                 nn2i_b[i1]  *(sum(nnoi_b[1:end-1] .* kvvs_d_n2_no[i1-1,:]));

  else

    RVTn2[i1] = nn2_b*(nn2i_b[i1+1]*kvt_down_n2[1,i1]+nn2i_b[i1-1]*kvt_up_n2[1,i1-1]-
                       nn2i_b[i1]*(kvt_up_n2[1,i1]+kvt_down_n2[1,i1-1]))+
                no2_b*(nn2i_b[i1+1]*kvt_down_n2[2,i1]+nn2i_b[i1-1]*kvt_up_n2[2,i1-1]-
                       nn2i_b[i1]*(kvt_up_n2[2,i1]+kvt_down_n2[2,i1-1]))+
                nno_b*(nn2i_b[i1+1]*kvt_down_n2[3,i1]+nn2i_b[i1-1]*kvt_up_n2[3,i1-1]-
                       nn2i_b[i1]*(kvt_up_n2[3,i1]+kvt_down_n2[3,i1-1]))+
                nn_b*(nn2i_b[i1+1]*kvt_down_n2[4,i1]+nn2i_b[i1-1]*kvt_up_n2[4,i1-1]-
                      nn2i_b[i1]*(kvt_up_n2[4,i1]+kvt_down_n2[4,i1-1]))+
                no_b*(nn2i_b[i1+1]*kvt_down_n2[5,i1]+nn2i_b[i1-1]*kvt_up_n2[5,i1-1]-
                      nn2i_b[i1]*(kvt_up_n2[5,i1]+kvt_down_n2[5,i1-1]));

    RVVn2[i1] = nn2i_b[i1+1]*sum(nn2i_b[1:end-1] .* kvv_down_n2[i1,:]) +
                nn2i_b[i1-1]*sum(nn2i_b[2:end]   .* kvv_up_n2[i1-1,:]) -
                nn2i_b[i1] *(sum(nn2i_b[2:end]   .* kvv_up_n2[i1,:]) +
                             sum(nn2i_b[1:end-1] .* kvv_down_n2[i1-1,:]));

    RVVsn2[i1] = nn2i_b[i1+1]*(sum(no2i_b[1:end-1] .* kvvs_d_n2_o2[i1,:])) +
                 nn2i_b[i1-1]*(sum(no2i_b[2:end]   .* kvvs_u_n2_o2[i1-1,:])) -
                 nn2i_b[i1]  *(sum(no2i_b[2:end]   .* kvvs_u_n2_o2[i1,:]) +
                               sum(no2i_b[1:end-1] .* kvvs_d_n2_o2[i1-1,:])) +
                 nn2i_b[i1+1]*(sum(nnoi_b[1:end-1] .* kvvs_d_n2_no[i1,:])) +
                 nn2i_b[i1-1]*(sum(nnoi_b[2:end]   .* kvvs_u_n2_no[i1-1,:])) -
                 nn2i_b[i1]  *(sum(nnoi_b[2:end]   .* kvvs_u_n2_no[i1,:]) +
                               sum(nnoi_b[1:end-1] .* kvvs_d_n2_no[i1-1,:]));
  end
end
for i2 = 1:l2

  RDo2[i2] = nn2_b*(no_b*no_b*kr_o2[1,i2]-no2i_b[i2]*kd_o2[1,i2]) +
             no2_b*(no_b*no_b*kr_o2[2,i2]-no2i_b[i2]*kd_o2[2,i2]) +
             nno_b*(no_b*no_b*kr_o2[3,i2]-no2i_b[i2]*kd_o2[3,i2]) +
             nn_b *(no_b*no_b*kr_o2[4,i2]-no2i_b[i2]*kd_o2[4,i2]) +
             no_b *(no_b*no_b*kr_o2[5,i2]-no2i_b[i2]*kd_o2[5,i2]);

  RZo2[i2] = sum(nnoi_b*no_b.*kb_o2[i2,:]-no2i_b[i2]*nn_b*kf_o2[i2,:]);

  if i2 == 1 # 0<->1

    RVTo2[i2] = nn2_b*(no2i_b[i2+1]*kvt_down_o2[1,i2] - no2i_b[i2]*kvt_up_o2[1,i2])+
                no2_b*(no2i_b[i2+1]*kvt_down_o2[2,i2] - no2i_b[i2]*kvt_up_o2[2,i2])+
                nno_b*(no2i_b[i2+1]*kvt_down_o2[3,i2] - no2i_b[i2]*kvt_up_o2[3,i2])+
                nn_b *(no2i_b[i2+1]*kvt_down_o2[4,i2] - no2i_b[i2]*kvt_up_o2[4,i2])+
                no_b *(no2i_b[i2+1]*kvt_down_o2[5,i2] - no2i_b[i2]*kvt_up_o2[5,i2]);

    RVVo2[i2] = no2i_b[i2+1]*sum(no2i_b[1:end-1] .* kvv_down_o2[i2,:]) -
                no2i_b[i2]  *sum(no2i_b[2:end]   .* kvv_up_o2[i2,:]);

    RVVso2[i2] = no2i_b[i2+1]*(sum(nn2i_b[1:end-1] .* kvvs_d_o2_n2[i2,:])) -
                 no2i_b[i2]  *(sum(nn2i_b[2:end]   .* kvvs_u_o2_n2[i2,:])) +
                 no2i_b[i2+1]*(sum(nnoi_b[1:end-1] .* kvvs_d_o2_no[i2,:])) -
                 no2i_b[i2]  *(sum(nnoi_b[2:end]   .* kvvs_u_o2_no[i2,:]));

  elseif i2 == l2 # Lmax <-> Lmax-1

    RVTo2[i2] = nn2_b*(no2i_b[i2-1]*kvt_up_o2[1,i2-1] - no2i_b[i2]*kvt_down_o2[1,i2-1])+
                no2_b*(no2i_b[i2-1]*kvt_up_o2[2,i2-1] - no2i_b[i2]*kvt_down_o2[2,i2-1])+
                nno_b*(no2i_b[i2-1]*kvt_up_o2[3,i2-1] - no2i_b[i2]*kvt_down_o2[3,i2-1])+
                nn_b *(no2i_b[i2-1]*kvt_up_o2[4,i2-1] - no2i_b[i2]*kvt_down_o2[4,i2-1])+
                no_b *(no2i_b[i2-1]*kvt_up_o2[5,i2-1] - no2i_b[i2]*kvt_down_o2[5,i2-1]);

    RVVo2[i2] = no2i_b[i2-1]*sum(no2i_b[2:end]   .* kvv_up_o2[i2-1,:]) -
                no2i_b[i2]  *sum(no2i_b[1:end-1] .* kvv_down_o2[i2-1,:]);

    RVVso2[i2] = no2i_b[i2-1]*(sum(nn2i_b[2:end]   .* kvvs_u_o2_n2[i2-1,:])) -
                 no2i_b[i2]  *(sum(nn2i_b[1:end-1] .* kvvs_d_o2_n2[i2-1,:])) +
                 no2i_b[i2-1]*(sum(nnoi_b[2:end]   .* kvvs_u_o2_no[i2-1,:])) -
                 no2i_b[i2]  *(sum(nnoi_b[1:end-1] .* kvvs_d_o2_no[i2-1,:]));

  else

    RVTo2[i2] = nn2_b*(no2i_b[i2+1]*kvt_down_o2[1,i2]+no2i_b[i2-1]*kvt_up_o2[1,i2-1]-
                       no2i_b[i2]*(kvt_up_o2[1,i2]+kvt_down_o2[1,i2-1]))+
                no2_b*(no2i_b[i2+1]*kvt_down_o2[2,i2]+no2i_b[i2-1]*kvt_up_o2[2,i2-1]-
                       no2i_b[i2]*(kvt_up_o2[2,i2]+kvt_down_o2[2,i2-1]))+
                nno_b*(no2i_b[i2+1]*kvt_down_o2[3,i2]+no2i_b[i2-1]*kvt_up_o2[3,i2-1]-
                       no2i_b[i2]*(kvt_up_o2[3,i2]+kvt_down_o2[3,i2-1]))+
                nn_b*(no2i_b[i2+1]*kvt_down_o2[4,i2]+no2i_b[i2-1]*kvt_up_o2[4,i2-1]-
                      no2i_b[i2]*(kvt_up_o2[4,i2]+kvt_down_o2[4,i2-1]))+
                no_b*(no2i_b[i2+1]*kvt_down_o2[5,i2]+no2i_b[i2-1]*kvt_up_o2[5,i2-1]-
                      no2i_b[i2]*(kvt_up_o2[5,i2]+kvt_down_o2[5,i2-1]));

    RVVo2[i2] = no2i_b[i2+1]*sum(no2i_b[1:end-1]  .* kvv_down_o2[i2,:]) +
                no2i_b[i2-1]*sum(no2i_b[2:end]    .* kvv_up_o2[i2-1,:]) -
                no2i_b[i2]  *(sum(no2i_b[2:end]   .* kvv_up_o2[i2,:]) +
                              sum(no2i_b[1:end-1] .* kvv_down_o2[i2-1,:]));

    RVVso2[i2] = no2i_b[i2+1]*(sum(nn2i_b[1:end-1] .* kvvs_d_o2_n2[i2,:])) +
                 no2i_b[i2-1]*(sum(nn2i_b[2:end]   .* kvvs_u_o2_n2[i2-1,:])) -
                 no2i_b[i2]  *(sum(nn2i_b[2:end]   .* kvvs_u_o2_n2[i2,:]) +
                               sum(nn2i_b[1:end-1] .* kvvs_d_o2_n2[i2-1,:])) +
                 no2i_b[i2+1]*(sum(nnoi_b[1:end-1] .* kvvs_d_o2_no[i2,:])) +
                 no2i_b[i2-1]*(sum(nnoi_b[2:end]   .* kvvs_u_o2_no[i2-1,:])) -
                 no2i_b[i2]  *(sum(nnoi_b[2:end]   .* kvvs_u_o2_no[i2,:]) +
                               sum(nnoi_b[1:end-1] .* kvvs_d_o2_no[i2-1,:]));
  end
end
for i3 = 1:l3

  RDno[i3] = nn2_b*(nn_b*no_b*kr_no[1,i3]-nnoi_b[i3]*kd_no[1,i3]) +
             no2_b*(nn_b*no_b*kr_no[2,i3]-nnoi_b[i3]*kd_no[2,i3]) +
             nno_b*(nn_b*no_b*kr_no[3,i3]-nnoi_b[i3]*kd_no[3,i3]) +
             nn_b *(nn_b*no_b*kr_no[4,i3]-nnoi_b[i3]*kd_no[4,i3]) +
             no_b *(nn_b*no_b*kr_no[5,i3]-nnoi_b[i3]*kd_no[5,i3]);

  RZno[i3] = sum(nn2i_b*no_b.*kf_n2[:,i3]-nnoi_b[i3]*nn_b*kb_n2[:,i3])+
             sum(no2i_b*nn_b.*kf_o2[:,i3]-nnoi_b[i3]*no_b*kb_o2[:,i3]);

  if i3 == 1 # 0<->1

    RVTno[i3] = nn2_b*(nnoi_b[i3+1]*kvt_down_no[1,i3] - nnoi_b[i3]*kvt_up_no[1,i3])+
                no2_b*(nnoi_b[i3+1]*kvt_down_no[2,i3] - nnoi_b[i3]*kvt_up_no[2,i3])+
                nno_b*(nnoi_b[i3+1]*kvt_down_no[3,i3] - nnoi_b[i3]*kvt_up_no[3,i3])+
                nn_b *(nnoi_b[i3+1]*kvt_down_no[4,i3] - nnoi_b[i3]*kvt_up_no[4,i3])+
                no_b *(nnoi_b[i3+1]*kvt_down_no[5,i3] - nnoi_b[i3]*kvt_up_no[5,i3]);

    RVVno[i3] = nnoi_b[i3+1]*sum(nnoi_b[1:end-1] .* kvv_down_no[i3,:]) -
                nnoi_b[i3]  *sum(nnoi_b[2:end]   .* kvv_up_no[i3,:]);

    RVVsno[i3] = nnoi_b[i3+1]*(sum(nn2i_b[1:end-1] .* kvvs_d_no_n2[i3,:])) -
                 nnoi_b[i3]  *(sum(nn2i_b[2:end]   .* kvvs_u_no_n2[i3,:])) +
                 nnoi_b[i3+1]*(sum(no2i_b[1:end-1] .* kvvs_d_no_o2[i3,:])) -
                 nnoi_b[i3]  *(sum(no2i_b[2:end]   .* kvvs_u_no_o2[i3,:]));

  elseif i3 == l3 # Lmax <-> Lmax-1

    RVTno[i3] = nn2_b*(nnoi_b[i3-1]*kvt_up_no[1,i3-1] - nnoi_b[i3]*kvt_down_no[1,i3-1])+
                no2_b*(nnoi_b[i3-1]*kvt_up_no[2,i3-1] - nnoi_b[i3]*kvt_down_no[2,i3-1])+
                nno_b*(nnoi_b[i3-1]*kvt_up_no[3,i3-1] - nnoi_b[i3]*kvt_down_no[3,i3-1])+
                nn_b *(nnoi_b[i3-1]*kvt_up_no[4,i3-1] - nnoi_b[i3]*kvt_down_no[4,i3-1])+
                no_b *(nnoi_b[i3-1]*kvt_up_no[5,i3-1] - nnoi_b[i3]*kvt_down_no[5,i3-1]);

    RVVno[i3] = nnoi_b[i3-1]*sum(nnoi_b[2:end]   .* kvv_up_no[i3-1,:]) -
                nnoi_b[i3]  *sum(nnoi_b[1:end-1] .* kvv_down_no[i3-1,:]);

    RVVsno[i3] = nnoi_b[i3-1]*(sum(nn2i_b[2:end]   .* kvvs_u_no_n2[i3-1,:])) -
                 nnoi_b[i3]  *(sum(nn2i_b[1:end-1] .* kvvs_d_no_n2[i3-1,:])) +
                 nnoi_b[i3-1]*(sum(no2i_b[2:end]   .* kvvs_u_no_o2[i3-1,:])) -
                 nnoi_b[i3]  *(sum(no2i_b[1:end-1] .* kvvs_d_no_o2[i3-1,:]));

  else

    RVTno[i3] = nn2_b*(nnoi_b[i3+1]*kvt_down_no[1,i3]+nnoi_b[i3-1]*kvt_up_no[1,i3-1]-
                       nnoi_b[i3]*(kvt_up_no[1,i3]+kvt_down_no[1,i3-1]))+
                no2_b*(nnoi_b[i3+1]*kvt_down_no[2,i3]+nnoi_b[i3-1]*kvt_up_no[2,i3-1]-
                       nnoi_b[i3]*(kvt_up_no[2,i3]+kvt_down_no[2,i3-1]))+
                nno_b*(nnoi_b[i3+1]*kvt_down_no[3,i3]+nnoi_b[i3-1]*kvt_up_no[3,i3-1]-
                       nnoi_b[i3]*(kvt_up_no[3,i3]+kvt_down_no[3,i3-1]))+
                nn_b*(nnoi_b[i3+1]*kvt_down_no[4,i3]+nnoi_b[i3-1]*kvt_up_no[4,i3-1]-
                      nnoi_b[i3]*(kvt_up_no[4,i3]+kvt_down_no[4,i3-1]))+
                no_b*(nnoi_b[i3+1]*kvt_down_no[5,i3]+nnoi_b[i3-1]*kvt_up_no[5,i3-1]-
                      nnoi_b[i3]*(kvt_up_no[5,i3]+kvt_down_no[5,i3-1]));

    RVVno[i3] = nnoi_b[i3+1]*sum(nnoi_b[1:end-1] .* kvv_down_no[i3,:]) +
                nnoi_b[i3-1]*sum(nnoi_b[2:end]   .* kvv_up_no[i3-1,:]) -
                nnoi_b[i3] *(sum(nnoi_b[2:end]   .* kvv_up_no[i3,:]) +
                             sum(nnoi_b[1:end-1] .* kvv_down_no[i3-1,:]));

    RVVsno[i3] = nnoi_b[i3+1]*(sum(nn2i_b[1:end-1] .* kvvs_d_no_n2[i3,:])) +
                 nnoi_b[i3-1]*(sum(nn2i_b[2:end]   .* kvvs_u_no_n2[i3-1,:])) -
                 nnoi_b[i3]  *(sum(nn2i_b[2:end]   .* kvvs_u_no_n2[i3,:]) +
                               sum(nn2i_b[1:end-1] .* kvvs_d_no_n2[i3-1,:])) +
                 nnoi_b[i3+1]*(sum(no2i_b[1:end-1] .* kvvs_d_no_o2[i3,:])) +
                 nnoi_b[i3-1]*(sum(no2i_b[2:end]   .* kvvs_u_no_o2[i3-1,:])) -
                 nnoi_b[i3]  *(sum(no2i_b[2:end]   .* kvvs_u_no_o2[i3,:]) +
                               sum(no2i_b[1:end-1] .* kvvs_d_no_o2[i3-1,:]));
  end
end

B = zeros(lall+4);
@fastmath @inbounds for i = 1:l1
  B[i] = RDn2[i] + RZn2[i] + RVTn2[i] + RVVn2[i] + RVVsn2[i];
end
@fastmath @inbounds for i = l1+1:l1+l2
  B[i] = RDo2[i] + RZo2[i] + RVTo2[i] + RVVo2[i] + RVVso2[i];
end
@fastmath @inbounds for i = l1+l2+1:lall
  B[i] = RDno[i] + RZno[i] + RVTno[i] + RVVno[i] + RVVsno[i];
end
B[lall+1] = - sum(RDno) - 2*sum(RDn2) - sum(RZn2) + sum(RZo2);
B[lall+2] = - sum(RDno) - 2*sum(RDo2) + sum(RZn2) - sum(RZo2);

mul!(dy,AA,B)

end

rpart_fho! (generic function with 1 method)

###### Solve the ODE system with the selected algorithm

In [7]:
prob = ODEProblem(rpart_fho!, Y0_bar, xspan, 1.)
#sol = DifferentialEquations.solve(prob, radau(), reltol=1e-8, abstol=1e-8, save_everystep=true, progress=true)
#sol = DifferentialEquations.solve(prob, RadauIIA(), reltol=1e-8, abstol=1e-8, save_everystep=true, progress=true)
sol  = DifferentialEquations.solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8, save_everystep=true, progress=true)
#sol = DifferentialEquations.solve(prob, Rosenbrock23(), reltol=1e-8, abstol=1e-8, save_everystep=true, progress=true)
#sol = DifferentialEquations.solve(prob, TRBDF2(), reltol=1e-8, abstol=1e-8, save_everystep=true, progress=true)
#sol = DifferentialEquations.solve(prob, ABDF2(), reltol=1e-8, abstol=1e-8, save_everystep=true, progress=true)
#sol = DifferentialEquations.solve(prob, Rodas5(), reltol=1e-8, abstol=1e-8, save_everystep=true, progress=true)
#sol = DifferentialEquations.solve(prob, Rodas4P(), reltol=1e-8, abstol=1e-8, save_everystep=true, progress=true)
#sol = DifferentialEquations.solve(prob, Rodas3(), reltol=1e-8, abstol=1e-8, save_everystep=true, progress=true)
#sol = DifferentialEquations.solve(prob, Kvaerno5(), reltol=1e-8, abstol=1e-8, save_everystep=true, progress=true)
#sol = DifferentialEquations.solve(prob, KenCarp4(), reltol=1e-8, abstol=1e-8, save_everystep=true, progress=true)

Z_rot = [1279.7235874117027 1778.4570686851096 3058.478143120314]

Kdr_n2 = [2.929873025050863e-24, 1.8577028977272204e-24, 1.1845025759941516e-24, 7.595004793920368e-25, 4.897251262592371e-25, 3.175477304999468e-25, 2.07060820825906e-25, 1.3577479044287071e-25, 8.95308366273324e-26, 5.936882261614623e-26, 3.95891813657228e-26, 2.654769982075115e-26, 1.7902330359211336e-26, 1.2140163272432249e-26, 8.278883663653682e-27, 5.6774239291747745e-27, 3.915283009034077e-27, 2.71523415051056e-27, 1.893580212435767e-27, 1.3279827011660632e-27, 9.365552887472857e-28, 6.64211996668733e-28, 4.737097262453685e-28, 3.3974273607681913e-28, 2.45030614960378e-28, 1.7771447861421297e-28, 1.2961568675907029e-28, 9.506586640000057e-29, 7.01171003237562e-29, 5.200625444702153e-29, 3.8789974768376236e-29, 2.909482233074258e-29, 2.1945436002904793e-29, 1.6645812928916605e-29, 1.2696911640380547e-29, 9.739204313449817e-30, 7.51244227605595e-30, 5.8273501035838884e-30, 4.545622497310215e-30, 3.56572554516584e-3

Matrix{Float64}

kd = [2.2801539825171678e-21 3.871911538104272e-21 6.521479763211337e-21 1.0894977816919883e-20 1.8053691810994088e-20 2.967325395309257e-20 4.8375314018223446e-20 7.822433786133641e-20 1.2546409810010582e-19 1.995981525186447e-19 3.1495828722078487e-19 4.92956999727185e-19 7.652873070403128e-19 1.1784182439562342e-18 1.7998399177712146e-18 2.7266398424523785e-18 4.097143245207786e-18 6.1065229630641274e-18 9.02747544501674e-18 1.3237260386818892e-17 1.925259977065172e-17 2.777410853420404e-17 3.974205762115299e-17 5.640531694460468e-17 7.940524935311644e-17 1.1087607885571364e-16 1.535627844359761e-16 2.1095682117014197e-16 2.8744889386506233e-16 3.8849656724064154e-16 5.208026728813944e-16 6.924982783220509e-16 9.133214686076186e-16 1.1947804548083825e-15 1.5502867219400876e-15 1.995241273639278e-15; 2.2801539825171678e-21 3.871911538104272e-21 6.521479763211337e-21 1.0894977816919883e-20 1.8053691810994088e-20 2.967325395309257e-20 4.8375314018223446e-20 7.822433786

Kdr_o2 = [2.210777992950407e-27, 1.6307227834228407e-27, 1.2085092569178865e-27, 8.998179658037094e-28, 6.731226044195709e-28, 5.0590448115160745e-28, 3.8201262306662293e-28, 2.898155859258801e-28, 2.2090248308694604e-28, 1.6916647273249798e-28, 1.3015559042271866e-28, 1.0061116908909433e-28, 7.813837957479249e-29, 6.097017506978973e-29, 4.77975151752964e-29, 3.7646797131366196e-29, 2.979103446759326e-29, 2.3685249857282418e-29, 1.8919305640943314e-29, 1.518333772974563e-29, 1.2242331851005124e-29, 9.917355453028588e-30, 8.071652327831858e-30, 6.60030266819586e-30, 5.422506630641923e-30, 4.4758051167831324e-30, 3.711736019528651e-30, 3.0925578377351846e-30, 2.5887696962100317e-30, 2.1772276344352467e-30, 1.839708843506883e-30, 1.5618135643330582e-30, 1.3321223687613734e-30, 1.1415472386257537e-30, 9.828301958219298e-31, 8.50154641848911e-31]
(36,)

Kdr_no = [3.731766682596349e-26, 2.5857607060482625e-26, 1.8015771208477198e-26, 1.26214154430503e-26, 8.891068252765757e-27, 6.29782377423

kd = [2.2801539825171678e-21 3.871911538104272e-21 6.521479763211337e-21 1.0894977816919883e-20 1.8053691810994088e-20 2.967325395309257e-20 4.8375314018223446e-20 7.822433786133641e-20 1.2546409810010582e-19 1.995981525186447e-19 3.1495828722078487e-19 4.92956999727185e-19 7.652873070403128e-19 1.1784182439562342e-18 1.7998399177712146e-18 2.7266398424523785e-18 4.097143245207786e-18 6.1065229630641274e-18 9.02747544501674e-18 1.3237260386818892e-17 1.925259977065172e-17 2.777410853420404e-17 3.974205762115299e-17 5.640531694460468e-17 7.940524935311644e-17 1.1087607885571364e-16 1.535627844359761e-16 2.1095682117014197e-16 2.8744889386506233e-16 3.8849656724064154e-16 5.208026728813944e-16 6.924982783220509e-16 9.133214686076186e-16 1.1947804548083825e-15 1.5502867219400876e-15 1.995241273639278e-15; 2.2801539825171678e-21 3.871911538104272e-21 6.521479763211337e-21 1.0894977816919883e-20 1.8053691810994088e-20 2.967325395309257e-20 4.8375314018223446e-20 7.822433786133641e-20 1.2546

kd = [1.0301568868392065e-24 1.9424164280759516e-24 3.634089004960486e-24 6.746259178225026e-24 1.2426382829542305e-23 2.2711231352936157e-23 4.118611981266768e-23 7.41097453085989e-23 1.3231650103627472e-22 2.3440503718282413e-22 4.1203500839864035e-22 7.186468717204215e-22 1.243687320215532e-21 2.1356059330560082e-21 3.638691831828028e-21 6.151537530730977e-21 1.031896941249319e-20 1.7175255929772458e-20 2.83651023088972e-20 4.648145737718437e-20 7.557695036260378e-20 1.2193074731897998e-19 1.9518720837153375e-19 3.100299960506758e-19 4.886189676847655e-19 7.641017140147065e-19 1.1856220589653044e-18 1.8253897728481104e-18 2.788554905562054e-18 4.2268519067637536e-18 6.357247344163824e-18 9.487142656740851e-18 1.4048048503660117e-17 2.0640053090193883e-17 3.00898392106527e-17 4.352543989000975e-17 6.247132480843365e-17 8.896772963160003e-17 1.257183130765698e-16 1.7627017141689917e-16 2.45229862025891e-16 3.385182801156742e-16 4.636658844925957e-16 6.301477712651145e-16 8.49755357675

...

kd = [2.9159524803908406e-22 5.21464323742581e-22 9.244445219228638e-22 1.624610284294871e-21 2.8302815214632736e-21 4.887898361443426e-21 8.368098629017667e-21 1.420180374539736e-20 2.3893089853158227e-20 3.9848613834789014e-20 6.588191494753632e-20 1.0797701125881768e-19 1.7543186625081095e-19 2.8255159128867413e-19 4.511273158620726e-19 7.140236297167713e-19 1.1203098058613917e-18 1.7425120218748364e-18 2.6867392614349593e-18 4.106646811346784e-18 6.222448027077576e-18 9.34646309112851e-18 1.3916992899591214e-17 2.0542609421389118e-17 3.005923371569749e-17 4.3602590711434205e-17 6.269873466191587e-17 8.937526680647484e-17 1.2629554145852556e-16 1.7691749437683623e-16 2.456776411957094e-16 3.3819920342096883e-16 4.615211625424213e-16 6.243422891546705e-16 8.372708625527283e-16 1.1130669719888802e-15 1.466860140089558e-15 1.916320791315839e-15 2.481760160770421e-15; 2.9159524803908406e-22 5.21464323742581e-22 9.244445219228638e-22 1.624610284294871e-21 2.8302815214632736e-21 4.8878983

kd = [1.057966718485492e-24 1.9932010029003947e-24 3.7260515312274295e-24 6.911388173012861e-24 1.272038871318275e-23 2.3230264187084767e-23 4.2094619056441585e-23 7.568637185490997e-23 1.3502912685994575e-22 2.3903192023666394e-22 4.1985855817638637e-22 7.317602062145381e-22 1.2654740259172358e-21 2.171482588810046e-21 3.697244198085227e-21 6.24623888202023e-21 1.047074809549682e-20 1.7416284173796203e-20 2.8744314886093475e-20 4.707248113345902e-20 7.648933532541437e-20 1.2332563284573098e-19 1.972988196872301e-19 3.1319461790412887e-19 4.933132293012017e-19 7.709920551844495e-19 1.1956269953589908e-18 1.8397558401397025e-18 2.8089456555943732e-18 4.255446673183322e-18 6.396842190010292e-18 9.541239331738462e-18 1.412090760889825e-17 2.0736674083135945e-17 3.0215811403690495e-17 4.3686584192437365e-17 6.267301469769666e-17 8.921374099344325e-17 1.2600900581093055e-16 1.7659975139856617e-16 2.4558241139103975e-16 3.388621793958189e-16 4.639461858325089e-16 6.302760236046369e-16 8.4959

Excessive output truncated after 524295 bytes.

, 6.620737471605673e-30, 5.438112319168522e-30, 4.487733579603324e-30, 3.720857578881506e-30, 3.0995319327142768e-30, 2.594097351120459e-30, 2.1812905244632576e-30, 1.842798468632659e-30, 1.5641531005857865e-30, 1.3338830824423952e-30, 1.1428608516366929e-30, 9.837982270346593e-31, 8.508554835063485e-31]
(36,)

Kdr_no = [3.776572895015312e-26, 2.6157453635131125e-26, 1.8217399365209347e-26, 1.2757648039917838e-26, 8.983553874575013e-27, 6.360907193438783e-27, 4.5288012770141606e-27, 3.242207395085332e-27, 2.3339505068103208e-27, 1.6894129350684188e-27, 1.2296267724124095e-27, 8.999204652522264e-28, 6.62259609567332e-28, 4.900560235843474e-28, 3.6463345901302563e-28, 2.72810213471767e-28, 2.0523813334842497e-28, 1.5525614886990161e-28, 1.1809537377342393e-28, 9.032548108736514e-29, 6.946739354889001e-29, 5.372111866617932e-29, 4.1773649032914676e-29, 3.266277672411158e-29, 2.5680123810147436e-29, 2.030179540940026e-29, 1.6138571960667645e-29, 1.289998133143001e-29, 1.0368272028261893e-2

LoadError: DomainError with -8.744945307302355e-202:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.

###### Post-processing

In [10]:
X    = sol.t; 
x_s  = X*Delta*100; 
Temp = sol[sum(l)+4,:]*T0;                                                                                                         
v    = sol[sum(l)+3,:]*v0;
Plots.pyplot()
f = plot(x_s,Temp);
plot!(title="Temperature vs X")
plot!(ylabel = "T[K]")
xlabel!("x [m]")
@show f

LoadError: UndefVarError: sol not defined